In [112]:
import os
from dotenv import load_dotenv
from langfuse.callback import CallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langfuse.callback import CallbackHandler
from langchain_openai import ChatOpenAI
import pandas as pd
import prompts as p
import time
import json

# Load the .env file
load_dotenv()

# Access the environment variables
os.environ["LANGFUSE_PUBLIC_KEY"] = os.environ.get("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.environ.get("LANGFUSE_SECRET_KEY")
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

handler = CallbackHandler()
handler.auth_check()

DB_PEDIA_PATH = "../Datasets/t2d_sm_nh/test/dbpedia_tables/{name}.csv"
WEBTABLES_PATH = "../Datasets/t2d_sm_nh/test/webtables/{name}.csv"
OUTPUT_PATH = "../Datasets/Evaluations/Sentiment_Analysis_Zero_shot_3.5_instruct.csv"
CORRESPONDENCES_PATH = "../Datasets/t2d_sm_nh/test/test_correspondences.csv"

MODEL = "gpt-3.5-turbo-instruct"
MAX_TOKENS = 15
TEMP = 0



In [113]:
# Read the CSV file into a DataFrame
df = pd.read_csv(CORRESPONDENCES_PATH)
df.rename(columns={'label': 'y_true'}, inplace=True)
df.rename(columns={'Unnamed: 0': 'table_index'}, inplace=True)

df['column_pred'] = None
df['prompt_name'] = None
df['prompt'] = None

# Delete all y_true values not equal to 1
df = df[df['y_true'] == 1]

df.head()

last = ""

for i in range (len(df)):
    if last != df.iloc[i]['table_name']:
        last = df.iloc[i]['table_name']
        print(f"Processing row {i}...")
        row = df.iloc[i]
        table_name = row['table_name']
        table_path = WEBTABLES_PATH.format(name=table_name)
        table = pd.read_csv(table_path)
        df_temp = df[df['table_name'] == table_name]
        for j in range (len(table.columns)):
            if j not in df_temp['column_index_left'].values:
                new_row = {'table_index': None, 'table_name': table_name, 'column_index_left': j, 'column_index_right' : -1, 'y_true': 0, 'column_pred': None, 'prompt_name': None, 'prompt': None}
                df = df._append(new_row, ignore_index=True)

        df_temp = df[df['table_name'] == table_name]
        print(df_temp.head())
           
            







Processing row 0...
    table_index                      table_name  column_index_left  \
0             0  29021592_3_2299138476894681059                  0   
1             7  29021592_3_2299138476894681059                  3   
209        None  29021592_3_2299138476894681059                  1   
210        None  29021592_3_2299138476894681059                  2   
211        None  29021592_3_2299138476894681059                  4   

     column_index_right  y_true column_pred prompt_name prompt  
0                     0       1        None        None   None  
1                     2       1        None        None   None  
209                  -1       0        None        None   None  
210                  -1       0        None        None   None  
211                  -1       0        None        None   None  
Processing row 2...
    table_index                      table_name  column_index_left  \
2            12  34899692_0_6530393048033763438                  0   
3        

In [114]:
from tabulate import tabulate

def table_format(df, type):
    num_columns = df.shape[1]
    for i in range(num_columns):
       df.rename(columns={ str(i) : 'Column ' + type + '-' + str(i)}, inplace=True)
    #print(df.head(5))
    random_rows = df.sample(n=5)
    df_destination = pd.DataFrame(random_rows)
    output = ''
    for column, values in df_destination.items():
      output += f"{column}: {' | '.join(map(str, values))}\n"
    return output





In [115]:
prompt_name = "zero_shot_general_prompt"
prompt = p.zero_shot_general_prompt
print(prompt)



input_variables=['Table_A', 'Table_B'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Description: Please identify the matching columns between Table A and Table B. For each column in Table A, specify the corresponding column in Table B. If a column in A has no corresponding column in Table B, you can map it to None. Represent each column mapping using a pair of column headers in a list, i.e., [Table A Column, Table B column or None]. Provide the mapping for each column in Table A and return all mappings in a list. Return the final result as JSON in the format {{"column_mappings": "<a list of column pairs>"}}.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['Table_A', 'Table_B'], template='Question:\nTable A:\n{Table_A}\n\nTable B:\n{Table_B}\n\nReturn the final result as JSON in the format {{"column_mappings": "<a list of column pairs>"}}.\nAnswer:'))]


In [119]:
def extract_column(s):
    if "None".lower() in s.lower():
        return -1
    else: 
        return int(s.split('-')[1])


In [125]:
def execute(df, prompt, prompt_name, start = 0):
    last = ""
    if(MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name = MODEL, temperature = TEMP, max_tokens=MAX_TOKENS, timeout=10)
    else:
        llm = ChatOpenAI(model_name = MODEL, temperature = TEMP, max_tokens=MAX_TOKENS, timeout=10)
    for i in range(start, len(df)):
        if last != df.iloc[i]['table_name']:
            last = df.iloc[i]['table_name']
            if(df["column_pred"][i] != None and df["column_pred"][i] != "error"):
                continue
            print(i)
            chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])

            print(df["table_name"][i])
            df_dbpedia = pd.read_csv(DB_PEDIA_PATH.format(name=df["table_name"][i]))
            df_webtables = pd.read_csv(WEBTABLES_PATH.format(name=df["table_name"][i]))
            db_pedia_table = table_format(df_dbpedia, 'A')
            webtales_table = table_format(df_webtables, 'B')
        
            try: 
                result = chain.run(Table_A = db_pedia_table, Table_B = webtales_table, callbacks=[handler])
            except Exception as e:
                print(e)
                for j in range(len(df_webtables.columns)):
                    cond = (df['table_name'] == last) & (df['column_index_left'] == j)
                    matching_rows = df[cond]
                    if not matching_rows.empty:
                        for index in matching_rows.index:
                            df.at[index, 'column_pred'] =  'error'
                            df.at[index, 'prompt_name'] = prompt_name
                            df.at[index, 'prompt'] = chain.prompt.format_prompt(Table_A = db_pedia_table, Table_B = webtales_table).text
                continue
            handler.langfuse.flush()
            prompt_text = chain.prompt.format_prompt(Table_A = db_pedia_table, Table_B = webtales_table).to_string()
            print(prompt_text)
            if (i == 2 ):
                break
            if (i < 5):
                print(prompt_text)
                print("\n")
                print(result)
                print("\n")
            
            if (i % 50 == 0):
                print(str(i) + " of  " + str(len(df)))

            ###TODO: Add y_pred
            data = json.loads(result)
            column_mappings = data["column_mappings"]
            
            for j in range(len(column_mappings)):
                cond = (df['table_name'] == last) & (df['column_index_left'] == j)
                matching_rows = df[cond]
                if not matching_rows.empty:
                    for index in matching_rows.index:
                        column_pred = extract_column(column_mappings[j][1])
                        df.at[index, 'column_pred'] =  column_pred
                        df.at[index, 'prompt_name'] = prompt_name
                        df.at[index, 'prompt'] = prompt_text
    return df




df = execute(df, prompt, prompt_name)
    



8
14380604_4_3329235705746762392


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


System: Description: Please identify the matching columns between Table A and Table B. For each column in Table A, specify the corresponding column in Table B. If a column in A has no corresponding column in Table B, you can map it to None. Represent each column mapping using a pair of column headers in a list, i.e., [Table A Column, Table B column or None]. Provide the mapping for each column in Table A and return all mappings in a list. Return the final result as JSON in the format {"column_mappings": "<a list of column pairs>"}.
Human: Question:
Table A:
Column A-0: Ford Motor Company | Continental Resources | EOG Resources | Denbury Resources | Crown Holdings
Column A-1: F | CLR | EOG | DEN | 1219601
Column A-2: William Clay Ford Jr., Jim Farley (businessman) | CEO, President (corporate title), Harold Hamm, Chairman, CFO | CEO, CFO | President (corporate title), CFO | CEO, CFO
Column A-3: 136300000000.0 | 5719000000.0 | 11032000000.0 | 1258000000.0 | 11500000000.0
Column A-4: 25700

JSONDecodeError: Expecting value: line 1 column 46 (char 45)

In [124]:
df.head(5)

,table_index,table_name,column_index_left,column_index_right,y_true,column_pred,prompt_name,prompt
0,0,29021592_3_2299138476894681059,0,0,1,1,zero_shot_general_prompt,System: Description: Please identify the match...
1,7,29021592_3_2299138476894681059,3,2,1,2,zero_shot_general_prompt,System: Description: Please identify the match...
2,12,34899692_0_6530393048033763438,0,0,1,1,zero_shot_general_prompt,System: Description: Please identify the match...
3,23,34899692_0_6530393048033763438,3,4,1,2,zero_shot_general_prompt,System: Description: Please identify the match...
4,26,55961337_0_6548713781034932742,5,0,1,-1,zero_shot_general_prompt,System: Description: Please identify the match...


In [9]:
df.to_csv(OUTPUT_PATH, index = True)